In [ ]:
import cv2
import matplotlib.pyplot as plt
import seaborn as sb
import numpy as np
import pandas as pd
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('../input/hubmap-organ-segmentation/train.csv')
df.head()

In [ ]:
df.info()

In [ ]:
sb.countplot(df['organ'])
plt.show()

In [ ]:
sb.countplot(df['sex'])
plt.show()

In [ ]:
sb.countplot(df['sex'])
plt.show()

In [ ]:
df['img_height'].value_counts()

In [ ]:
sb.countplot(x = df['organ'], hue=df['sex'])
plt.show()

In [ ]:
sb.distplot(x = df['age'])
plt.show()

In [ ]:
def helper(mask, img_shape):
  
    canvas = np.zeros(img_shape).T
    canvas[tuple(zip(*mask))] = 1.0

      # This is the Equivalent for loop of the above command for better understanding.
      # for pos in range(len(p_loc)):
      #   canvas[pos[0], pos[1]] = 1

    return canvas

def get_mask(rle_string, img_shape):
    rle = [int(i) for i in rle_string.split(' ')]
    pairs = list(zip(rle[0::2],rle[1::2]))

    p_loc = []

    for start, length in pairs:
        for p_pos in range(start, start + length):
            p_loc.append((p_pos % img_shape[1], p_pos // img_shape[0]))
  
    return helper(p_loc, img_shape)

In [ ]:
df['image_path'] = '../input/hubmap-organ-segmentation/train_images/'
df['image_path'] = df['image_path'].str.cat(df['id'].astype(str))
df['image_path'] = df['image_path'] + '.tiff'
df.head()

In [ ]:
for part in df['organ'].unique():

  x = df[df['organ'] == part]
  index_list = x.index
  idx = index_list[np.random.randint(0, x.shape[0])]

  class_of_scan = df.loc[idx, 'organ']
  image_path = df.loc[idx, 'image_path']
  id = df.loc[idx, 'id'] 
  
  image = np.array(Image.open(image_path)) / 255
  k = (df.loc[idx, 'img_height'], df.loc[idx, 'img_width'])

  rle_string = df.loc[idx, 'rle']
  mask = get_mask(rle_string, k)


  fig, ax = plt.subplots(1,3, figsize=(15,15))
  ax[0].set_title(f'Image : {id}')
  ax[0].imshow(image)

  ax[1].set_title(f'Mask : {id}')
  ax[1].imshow(mask)

  ax[2].set_title(f'{class_of_scan} Segmented : {id}')
  ax[2].imshow(np.dstack((mask, np.zeros(mask.shape), np.argmax(image, axis=-1))))
  plt.show()